In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from scipy.stats import norm
from sklearn.ensemble import IsolationForest
from wordcloud import WordCloud
import warnings
warnings.simplefilter(action="ignore")

In [3]:
data=pd.read_csv("ds_salaries.csv")

In [4]:
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [5]:
data.shape

(3755, 11)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


In [7]:
data.nunique()

work_year                4
experience_level         4
employment_type          4
job_title               93
salary                 815
salary_currency         20
salary_in_usd         1035
employee_residence      78
remote_ratio             3
company_location        72
company_size             3
dtype: int64

In [8]:
data.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [9]:
data.duplicated().sum()

1171

In [10]:
data = data.drop_duplicates()

In [11]:
data.duplicated().sum()

0

In [12]:
data.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,2584.000000,2.584000e+03,2584.000000,2584.000000
mean,2022.301084,2.103653e+05,133409.280186,50.483746
std,0.749179,8.080375e+05,67136.837329,48.163707
min,2020.000000,6.000000e+03,5132.000000,0.000000
25%,2022.000000,9.000000e+04,84975.000000,0.000000
50%,2022.000000,1.346300e+05,130000.000000,50.000000
75%,2023.000000,1.825625e+05,175000.000000,100.000000
max,2023.000000,3.040000e+07,450000.000000,100.000000


In [13]:
numerical_columns=data.select_dtypes(include=['int64']).columns
categorical_columns=data.select_dtypes(include=['object']).columns

In [14]:
numerical_columns

Index(['work_year', 'salary', 'salary_in_usd', 'remote_ratio'], dtype='object')

In [15]:
categorical_columns

Index(['experience_level', 'employment_type', 'job_title', 'salary_currency',
       'employee_residence', 'company_location', 'company_size'],
      dtype='object')

In [16]:
iso_forest=IsolationForest(contamination=0.05, random_state=42)
outliers = iso_forest.fit_predict(data[numerical_columns])
outlier_count = (outliers == -1).sum()
total_count = len(outliers)
outlier_percentage = (outlier_count / total_count) * 100
print(f"Percentage of numerical outliers: {outlier_percentage:.2f}%")

Percentage of numerical outliers: 5.03%


In [17]:
cat_outliers = {}
cat_outlier_count = 0

for col in categorical_columns:
    value_counts = data[col].value_counts()
    rare_categories = value_counts[value_counts < 0.05 * len(data)]
    cat_outliers[col] = rare_categories.index.tolist()
    cat_outlier_count += data[col].isin(rare_categories.index).sum()

total_count = len(data)

categorical_outlier_percentage = (cat_outlier_count / total_count) * 100

print(f"Categorical outliers count: {cat_outlier_count}")
print(f"Categorical outliers percentage: {categorical_outlier_percentage:.2f}%")

Categorical outliers count: 2153
Categorical outliers percentage: 83.32%


In [18]:
# Identify outliers in categorical columns
cat_outliers = {}

for col in categorical_columns:
    value_counts = data[col].value_counts()
    rare_categories = value_counts[value_counts < 0.05 * len(data)]
    cat_outliers[col] = rare_categories.index.tolist()

# Impute outliers with the mode of each column
for col, outliers in cat_outliers.items():
    mode_value = data[col].mode()[0]  # Get the mode of the column
    data[col] = data[col].apply(lambda x: mode_value if x in outliers else x)

# Verify imputation
print(f"Data shape after imputing categorical outliers: {data.shape}")

Data shape after imputing categorical outliers: (2584, 11)


In [19]:
def assign_broader_category(job_title):
    data_engineering = ["Data Engineer", "Data Analyst", "Analytics Engineer", "BI Data Analyst", "Business Data Analyst", "BI Developer", "BI Analyst", "Business Intelligence Engineer", "BI Data Engineer", "Power BI Developer"]
    data_scientist = ["Data Scientist", "Applied Scientist", "Research Scientist", "3D Computer Vision Researcher", "Deep Learning Researcher", "AI/Computer Vision Engineer"]
    machine_learning = ["Machine Learning Engineer", "ML Engineer", "Lead Machine Learning Engineer", "Principal Machine Learning Engineer"]
    data_architecture = ["Data Architect", "Big Data Architect", "Cloud Data Architect", "Principal Data Architect"]
    management = ["Data Science Manager", "Director of Data Science", "Head of Data Science", "Data Scientist Lead", "Head of Machine Learning", "Manager Data Management", "Data Analytics Manager"]
    
    if job_title in data_engineering:
        return "Data Engineering"
    elif job_title in data_scientist:
        return "Data Science"
    elif job_title in machine_learning:
        return "Machine Learning"
    elif job_title in data_architecture:
        return "Data Architecture"
    elif job_title in management:
        return "Management"
    else:
        return "Other"

data['job_category'] = data['job_title'].apply(assign_broader_category)


In [20]:
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_category
0,2023,SE,FT,Data Engineer,80000,EUR,85847,US,100,US,L,Data Engineering
1,2023,MI,FT,Data Engineer,30000,USD,30000,US,100,US,S,Data Engineering
2,2023,MI,FT,Data Engineer,25500,USD,25500,US,100,US,S,Data Engineering
3,2023,SE,FT,Data Scientist,175000,USD,175000,US,100,US,M,Data Science
4,2023,SE,FT,Data Scientist,120000,USD,120000,US,100,US,M,Data Science


In [21]:
data.drop(['salary','salary_currency','job_title'],axis=1,inplace= True)

In [22]:
data.head()

,work_year,experience_level,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_category
0,2023,SE,FT,85847,US,100,US,L,Data Engineering
1,2023,MI,FT,30000,US,100,US,S,Data Engineering
2,2023,MI,FT,25500,US,100,US,S,Data Engineering
3,2023,SE,FT,175000,US,100,US,M,Data Science
4,2023,SE,FT,120000,US,100,US,M,Data Science


In [23]:
data = pd.get_dummies(data,columns=['experience_level', 'employment_type', 'employee_residence', 'company_location', 'company_size','job_category'],dtype=int)

In [24]:
data.head()

,work_year,salary_in_usd,remote_ratio,experience_level_EN,experience_level_MI,experience_level_SE,employment_type_FT,employee_residence_GB,employee_residence_US,company_location_GB,company_location_US,company_size_L,company_size_M,company_size_S,job_category_Data Engineering,job_category_Data Science,job_category_Machine Learning
0,2023,85847,100,0,0,1,1,0,1,0,1,1,0,0,1,0,0
1,2023,30000,100,0,1,0,1,0,1,0,1,0,0,1,1,0,0
2,2023,25500,100,0,1,0,1,0,1,0,1,0,0,1,1,0,0
3,2023,175000,100,0,0,1,1,0,1,0,1,0,1,0,0,1,0
4,2023,120000,100,0,0,1,1,0,1,0,1,0,1,0,0,1,0


In [25]:
X = data.drop('salary_in_usd',axis=1)
y = data['salary_in_usd']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import numpy as np

# Define models and hyperparameters for tuning
models = {
    'Linear Regression': (LinearRegression(), {}),
    'SVR': (SVR(), {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.2, 0.5]}),
    'Random Forest': (RandomForestRegressor(random_state=42), {'n_estimators': [50, 100, 200]}),
    'AdaBoost': (AdaBoostRegressor(random_state=42), {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 1]}),
    'XGBoost': (XGBRegressor(random_state=42), {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 0.2]}),
    'Gradient Boosting': (GradientBoostingRegressor(random_state=42), {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5]}),
}

# Function to calculate MAPE
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Loop through models and perform hyperparameter tuning
for name, (model, params) in models.items():
    # Apply GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Best estimator from grid search
    best_model = grid_search.best_estimator_
    
    # Training set predictions
    y_train_pred = best_model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    rmse_train = np.sqrt(train_mse)
    mape_train = calculate_mape(y_train, y_train_pred)
    
    # Test set predictions
    y_test_pred = best_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    rmse_test = np.sqrt(test_mse)
    mape_test = calculate_mape(y_test, y_test_pred)
    
    # Print results
    print(name)
    print('=' * len(name))
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Train Mean Squared Error: {train_mse}")
    print(f"Train R-squared: {train_r2}")
    print(f"Train RMSE: {rmse_train:.2f}")
    print(f"Train MAPE: {mape_train:.2f}%")
    print('\n')
    print(f"Test Mean Squared Error: {test_mse}")
    print(f"Test R-squared: {test_r2}")
    print(f"Test RMSE: {rmse_test:.2f}")
    print(f"Test MAPE: {mape_test:.2f}%")
    print('\n\n')


Linear Regression
Best Parameters: {}
Train Mean Squared Error: 3376315056.0640674
Train R-squared: 0.2560530643877489
Train RMSE: 58106.07
Train MAPE: 60.13%


Test Mean Squared Error: 3190273045.5381217
Test R-squared: 0.27064000490443274
Test RMSE: 56482.50
Test MAPE: 62.73%



SVR
===
Best Parameters: {'C': 10, 'epsilon': 0.1}
Train Mean Squared Error: 4551122514.785996
Train R-squared: -0.0028073779400037413
Train RMSE: 67462.01
Train MAPE: 82.70%


Test Mean Squared Error: 4381719918.864449
Test R-squared: -0.0017484939111414022
Test RMSE: 66194.56
Test MAPE: 87.98%



Random Forest
Best Parameters: {'n_estimators': 200}
Train Mean Squared Error: 2898921133.2076344
Train R-squared: 0.3612434095100363
Train RMSE: 53841.63
Train MAPE: 50.35%


Test Mean Squared Error: 3484393134.75165
Test R-squared: 0.20339829118142994
Test RMSE: 59028.75
Test MAPE: 59.80%



AdaBoost
Best Parameters: {'learning_rate': 0.01, 'n_estimators': 100}
Train Mean Squared Error: 3392259786.7446237
Train R